In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
import numpy as np
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline as IMBPipeline
from sklearn.feature_selection import RFE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import extract_features

In [2]:
def get_classification_report(y_test, y_pred):
    '''Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format'''
    report = classification_report(y_test, y_pred, output_dict=True)
    df_classification_report = pd.DataFrame(report).transpose()
    df_classification_report = df_classification_report.sort_values(by=['f1-score'], ascending=False)
    return df_classification_report

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/semcovici/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
path_raw_data = '../data/raw/'
path_processed_data = '../data/processed/'

In [5]:
list_target = ['ig','bo', 'cl', 'co', 'gl', 'lu']

In [6]:
corpus = 'ig'
model_name = 'facebook/fasttext-pt-vectors'
model_name = 'neuralmind/bert-base-portuguese-cased'

In [13]:
df_results = pd.DataFrame({})


for target in list_target:


    path_data_train = path_processed_data + f'train_r3_{target}_top_mentioned_timelines_{model_name.replace("/", "_")}.parquet'
    path_data_test = path_processed_data + f'test_r3_{target}_top_mentioned_timelines_{model_name.replace("/", "_")}.parquet'

    data_train = pd.read_parquet(path_data_train)
    data_test = pd.read_parquet(path_data_test)

    X_train = data_train[[col for col in data_train.columns if 'emb' in col]]
    y_train = data_train.Polarity.map({'against': 0, 'for': 1})
    
    X_test = data_test[[col for col in data_test.columns if 'emb' in col]]
    y_test = data_test.Polarity.map({'against': 0, 'for': 1})


    pipe = IMBPipeline(
        steps = [
            ('scaling', MaxAbsScaler()),
            ('selection', None),
            ('estimator', XGBClassifier(
                random_state = 42,
                verbosity = 3,
                device = 'cuda',
                tree_method = 'hist'
                ))
        ],
        verbose = True
        )

    print('Training ...')
    pipe_trained = pipe.fit(X_train, y_train)

    y_pred = pipe_trained.predict(X_test)
    y_pred_proba = pipe_trained.predict_proba(X_test)
    
    df_classification_report = get_classification_report(y_test, y_pred)
    
    df_classification_report = df_classification_report.reset_index().rename(columns = {"index": "class"})
    
    df_classification_report['corpus'] = corpus

    df_results = pd.concat([df_results, df_classification_report])

NameError: name 'target' is not defined

In [12]:
print(display(df_results[df_results['class'] == 'macro avg'].sort_values('f1-score')))

,class,precision,recall,f1-score,support,corpus
3,macro avg,0.682796,0.516144,0.498358,188.0,bo
3,macro avg,0.604328,0.602307,0.600649,574.0,cl


None
